In [1]:
import os
import time
import datetime
import selenium.webdriver as webdriver
import chromedriver_binary
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementNotInteractableException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'
PATH = 'C:\Program Files\Mozilla Firefox'
driver = webdriver.Firefox(executable_path=r'C:\Program Files\Mozilla Firefox\geckodriver.exe')

C:\Users\antho\AppData\Local\Temp\ipykernel_22576\2561988501.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=r'C:\Program Files\Mozilla Firefox\geckodriver.exe')


In [3]:
def xpath_element(xpath):
    try:
        element = driver.find_element(By.XPATH, xpath)
    except NoSuchElementException:
        element = []
    return element

In [4]:
def real_time_price(stock_code):
    url  = 'https://finance.yahoo.com/quote/'+stock_code+'?p='+stock_code+'&.tsrc=fin-srch'
    driver.get(url)
    t_stamp = datetime.datetime.now()
    t_stamp = t_stamp.strftime('%Y-%m-%d %H:%M:%S')
    output_path =  str(t_stamp[0:11]) + 'crypto_data.xlsx'
    xlwriter = pd.ExcelWriter(output_path)
    ########## PRICE AND CHANGES ###########
    xpath = '//*[@id="quote-header-info"]/div[3]/div[1]/div[1]'
    stock_price_info = xpath_element(xpath)
#     print(stock_price_info.text.split()[0], stock_price_info.text.split()[1])
    for stock in stock_price_info:
        if stock != []:
            stock_price_temp = stock.text.split()[0]
            if stock_price_temp.find('+') != -1:
                price = stock_price_temp.split('+')[0]
                try:
                    change = '+' + stock_price_temp.split('+')[1] + ' ' + stock.text.split()[1]
                except IndexError:
                    change = []
            elif stock_price_temp.find('-') != -1:
                price = stock_price_temp.split('-')[0]
                try:
                    change = '-' + stock_price_temp.split('-')[1] + ' ' + stock.text.split()[1]
                except IndexError:
                    change = []
            else:
                price,change = [],[]
        else:
            price, change = [],[]
        ########## Volume ###########
        xpath = '//*[@id="quote-summary"]/div[2]/table'
        volume_temp = xpath_element(xpath)
    #     print(volume_temp.text.split()[-9])
    for vol in volume_temp:
        if vol != []:
    #         volume = volume_temp.text.split()[-9]
            for i, text in enumerate(vol.text.split()):
                if text == 'Volume':
                    volume = vol.text.split()[i+1]
                    break
                else:
                    volume = []
        return price, change, volume

In [5]:
crypto_stonk = ['XRP', 'DOGE', 'SHIB', 'LINK', 'AXS', 'XLM', 'TRX',
 'RVN', 'ADA', 'AVAX', 'ATOM', 'ALGO', 'DOT', 'XMR', 'ICP', 'ETC', 'MANA',
 'AAVE', 'LUNA', 'SOL', 'BTC', 'LTC', 'ETH', 'APE', 'BNB', 'ONE', 'TEL',
'ERG', 'RTM']

In [6]:
stonks = []
for i in (crypto_stonk):
    stonks.append(str(i) + '-USD')

In [8]:
info = []
time_stamp = datetime.datetime.now()
time_stamp = time_stamp.strftime('%Y-%m-%d %H:%M:%S')

for stock_code in stonks:
    price, change, volume = real_time_price(stock_code)
    info.columns('Time','Price','Change','Volume')
    info.append(price)
    info.extend([change])
    info.extend([volume])
col = [time_stamp]
col.extend(info)
df = pd.DataFrame(col)
df = df.T

print(col)
driver.close()

WebDriverException: Message: Failed to write request to stream
